In [1]:
# External imports
import pandas as pd
import numpy as np
import os

# Set the working directory
os.chdir(r'C:\Users\hp\VSCode_projects\bidding_strategy')

# Internal imports
from logger.logging import getLogger

logger = getLogger()

# Read the CSV file
df = pd.read_csv('interview_test_data.csv')
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'interview_test_data.csv'

In [4]:
# External imports
import yaml
import pandas as pd
from sklearn.model_selection import train_test_split

# Internal imports
from bidding_utils.data_preprocessing import DataPreprocessor, FeatureEngineer
from bidding_utils.modelling import TwoStageModel
from logger.logging import getLogger

logger = getLogger()


def load_config():
    """
    Load configuration from the YAML file.

    Returns
    -------
    dict
        Configuration parameters.
    """
    with open("configs/config.yaml", "r") as f:
        return yaml.safe_load(f)



ModuleNotFoundError: No module named 'imblearn'

In [3]:
df.head(10)

,connection_type,language,imp_time,device_height,device_model,device_os_version,device_width,age_group,carrier,city,...,app_activity__view_detail_60d,app_activity__view_listing_60d,nb_of_transactions,action_value_usd,transaction_timestamps,publisher_app_id,bid_request_id,supplier_id,creative_id,deviceId
0,cellular,de,1718895782,1633.0,galaxy a53 5g,14,810.0,NaN,t-mobile,Hamburg,...,13,0,0,0.0,NaN,6fef01426c277ee6fdc8b0be1fbba138e047021b5326bb...,2ebd884107aab56cff719dd87be72d959c52f0537bb25c...,da70dfa4d9f95ac979f921e8e623358236313f334afcd0...,eb3009287af9534bbb4b9ba703b201878caa766db7934e...,188ce07a78530499ca7ea06930452c9f229ed37ba32aa8...
1,wifi,de,1718895933,1928.0,小新 pad gt,12,1200.0,NaN,NaN,Duisburg,...,0,0,0,0.0,NaN,af232c76f0c137eea5e074d2a902d16f18d5305becbe1c...,df136720b7322a33b800bd5c987b42cfa0fa622e52ed23...,da70dfa4d9f95ac979f921e8e623358236313f334afcd0...,dc826ac1697a10d3dc9cdeddc217290d1604911b6ccd5b...,dc4f93f4f8a14d13ed616213a78c1179b344d9639031b8...
2,wifi,de,1718876697,722.0,galaxy a71,13.0,360.0,NaN,o2,Berlin,...,2,0,0,0.0,NaN,d6b2988423f73cc31d49d6d77687d958261334f237e9f4...,eac3c40c48319a0efd790f655088e4d0828e58fdcab28e...,76ebdb6d45c61ca12e622118cc90939ade672adf7890aa...,441272beebd6e9f4826c41cb407a9d18ec2da2425a1ff9...,e3a94e6c1895041f9b417f58395233677866655e3e0d8a...
3,wifi,de,1718892256,2217.0,sm-s911b,14,1080.0,NaN,t-mobile,Limburg an der Lahn,...,9,0,0,0.0,NaN,d51063ea952479e9eaf0390b3c3a3283837c59ea501b26...,840749de0e2127b2e4a90da3660c2ff9fe70aa746ed2de...,da70dfa4d9f95ac979f921e8e623358236313f334afcd0...,dc826ac1697a10d3dc9cdeddc217290d1604911b6ccd5b...,f21a8ee39f8fb215eb3c86aaf95b6ef3c7b6ba855b8c68...
4,wifi,de,1718876359,NaN,NaN,14.0,NaN,NaN,NaN,Versmold,...,20,0,0,0.0,NaN,ef877db092e884b067fe1a1c82750ac22ec54cad2a591b...,69ef04f1e8e5bef13941dad255fac0b888fcbefa635a8b...,02e6295d8f522840f09b5194b3f023799ad6ed3306d929...,c3e99cbd681035ab74a7047a71c555af68a09d4cc2cc5c...,8009e3cb3916ce59aa92113d9be7bb1bf0af0f10f71b17...
5,wifi,de,1718861357,2280.0,galaxy s10e,12.0.0,1080.0,NaN,NaN,ludwigsburg,...,11,0,0,0.0,NaN,ef877db092e884b067fe1a1c82750ac22ec54cad2a591b...,6c0fd89dc7bb5364df36cafdfbdc0da4c34ed4d8df8c42...,06b2d82840e43ed8432b3f444de18b57dbe60637c99379...,dc826ac1697a10d3dc9cdeddc217290d1604911b6ccd5b...,547b06f4bc27457b4860e22e080a7f5f00f14315adca9f...
6,wifi,de,1718892990,2340.0,galaxys23+,14,1080.0,NaN,NaN,cologne,...,32,0,0,0.0,NaN,7fd56aaf1535ec12083f531ef3fbf64fc9e84513e42022...,006c60d1bac4393a3e208319c2bc74ba903ab773cbf2bd...,06b2d82840e43ed8432b3f444de18b57dbe60637c99379...,dc826ac1697a10d3dc9cdeddc217290d1604911b6ccd5b...,0cb67b1b1de0a02a4830d58cabc913211dfb7452ec5818...
7,wifi,de,1718864214,792.0,xiaomi m2101k6g,12.0,393.0,NaN,NaN,Saarbr�cken,...,1,0,0,0.0,NaN,23d1148586d0b90f901129cd63f4ef451b08b99115ffec...,df14313919ec8ee88579961f7d6e0c47f145d227454509...,76ebdb6d45c61ca12e622118cc90939ade672adf7890aa...,441272beebd6e9f4826c41cb407a9d18ec2da2425a1ff9...,be10d57aeb2ec3d54d0632a50d00445d0952b44c24bc05...
8,cellular,de,1718881100,1520.0,huawei amn-lx9,9.0.0,720.0,NaN,NaN,munich,...,27,0,0,0.0,NaN,3e46b03423b7c58fefcf2971d1c7b4ec2ad127153e89f8...,e300865f09e65a5654e99e71db0768950a0375f600f5c3...,06b2d82840e43ed8432b3f444de18b57dbe60637c99379...,dc826ac1697a10d3dc9cdeddc217290d1604911b6ccd5b...,1f8dff1ab305bc4aabe9c9593eecc12a6aad834a54df1c...
9,wifi,de,1718902234,2466.0,redmi 23090ra98g,13,1220.0,NaN,o2,Nuremberg,...,8,0,0,0.0,NaN,ec8829d0a1dd8962fe8dfa216627d80915823e8bea7549...,76fe831011eb52bbdf9c540681de70af4e9b5175359852...,02e6295d8f522840f09b5194b3f023799ad6ed3306d929...,dc826ac1697a10d3dc9cdeddc217290d1604911b6ccd5b...,a9b21494492098746a441725e71c1a422a18589e2c54c1...


In [ ]:
# Preprocess timestamps columns
def process_timestamps(timestamps):
    if pd.isna(timestamps):
        return []
    if isinstance(timestamps, str):
        # Split multiple timestamps
        timestamps_list = timestamps.split(';')
    else:
        # Handle single timestamp
        timestamps_list = timestamps
    
    # Convert to float and then to integer (if needed)
    timestamps_list = [int(float(ts)) for ts in timestamps_list]
    # Sort the timestamps
    timestamps_list.sort()
    
    # Convert back to single element if only one element
    if len(timestamps_list) == 1:
        timestamps_list = timestamps
    return timestamps_list

df['processed_timestamps'] = df['transaction_timestamps'].apply(process_timestamps)

# Create a new column with the number of transactions
df['last_transaction_time'] = df['processed_timestamps'].apply(lambda x: x[-1] if isinstance(x, list) and x else np.nan)

# drop the original and preprocessed timestamp columns
df.drop(['transaction_timestamps', 'processed_timestamps'], axis=1, inplace=True)

# Define recency and latency features
df["time_since_last_transaction"] = df["last_transaction_time"] - df["imp_time"]
df["click_latency"] = df["click_time"] - df["imp_time"]

print(df[['imp_time', 'last_transaction_time', 'time_since_last_transaction', 'click_latency']])

In [27]:
# Identify the most important features using chi-square test
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Separate features and target
X = df.drop(['click_event', 'click_time', 'imp_time'], axis=1)
y = df['click_event']

# Encode categorical features
categorical_features = X.columns.tolist()
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Create a pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(score_func=chi2, k='all'))
])

# Fit the pipeline
pipeline.fit(X, y)

# Get the scores
scores = pipeline.named_steps['selector'].scores_

# Create a DataFrame with feature names and their scores
feature_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
feature_scores = pd.DataFrame({'Feature': feature_names, 'Score': scores})

# Sort features by score
feature_scores = feature_scores.sort_values(by='Score', ascending=False)

print(feature_scores)

                                                  Feature         Score
841577                                  click_latency_nan  1.002740e+05
5902                                 nb_of_transactions_1  4.064470e+04
78                                          format_320x50  2.183711e+04
72                            creative_type_video_480x640  1.993471e+04
80                                         format_480x640  1.993471e+04
...                                                   ...           ...
4051                  app_activity__additional_2_60d_1316  1.416769e-05
19770   publisher_app_id_853bf25c3eda320ad455ff0766918...  1.416769e-05
4767                           app_activity__open_60d_149  1.381859e-05
3223                   app_activity__additional_2_60d_488  8.912664e-07
553                               imp_activity__imp_1d_46  1.618586e-09

[841578 rows x 2 columns]


In [28]:
# Select only numerical features
X_numerical = X.select_dtypes(include=[np.number])

# Compute the correlation matrix
corr_matrix = X_numerical.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.8
threshold = 0.8
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# Drop highly correlated features
X_reduced = X_numerical.drop(columns=to_drop)

In [29]:
print("\nFeatures to drop:", to_drop)


Features to drop: ['app_activity__additional_3_60d', 'app_activity__reattribution_60d', 'app_activity__signup_60d']


In [30]:
corr_matrix

,imp_activity__click_1d,imp_activity__imp_1d,imp_activity__click_7d,imp_activity__imp_7d,imp_activity__click_90d,imp_activity__imp_90d,app_activity__inactivity_days,app_activity__add_to_basket_3d,app_activity__additional_1_3d,app_activity__additional_2_3d,...,app_activity__reattribution_60d,app_activity__signup_60d,app_activity__transaction_60d,app_activity__view_detail_60d,app_activity__view_listing_60d,nb_of_transactions,action_value_usd,last_transaction_time,time_since_last_transaction,click_latency
imp_activity__click_1d,1.000000,0.375894,0.520185,0.445284,0.789850,0.653726,0.106074,0.009005,0.038830,0.041497,...,0.655670,0.187022,0.200922,0.037706,0.025901,0.014698,0.013834,0.123035,0.160557,0.000017
imp_activity__imp_1d,0.375894,1.000000,0.181649,0.273512,0.258126,0.346763,0.052385,0.003625,0.015006,0.022386,...,0.237542,0.076979,0.087520,0.022595,0.009904,0.006050,0.007196,0.108050,0.111058,0.002622
imp_activity__click_7d,0.520185,0.181649,1.000000,0.555532,0.702000,0.500713,0.147846,0.000231,0.034796,0.020378,...,0.632464,0.058959,0.185524,0.001450,0.002040,0.006967,0.008601,0.166096,0.203158,0.000012
imp_activity__imp_7d,0.445284,0.273512,0.555532,1.000000,0.532240,0.726335,0.114799,0.002255,0.039863,0.023909,...,0.498782,0.065756,0.235656,0.031835,0.000354,0.005804,0.005718,0.119376,0.178130,0.001395
imp_activity__click_90d,0.789850,0.258126,0.702000,0.532240,1.000000,0.759641,0.122578,0.005120,0.041960,0.037795,...,0.806021,0.168446,0.217444,0.016798,0.015250,0.014255,0.015573,0.151314,0.184132,0.001023
imp_activity__imp_90d,0.653726,0.346763,0.500713,0.726335,0.759641,1.000000,0.114450,0.005223,0.047189,0.043996,...,0.654850,0.176162,0.278639,0.041278,0.008903,0.002564,0.001077,0.117637,0.153024,0.002323
app_activity__inactivity_days,0.106074,0.052385,0.147846,0.114799,0.122578,0.114450,1.000000,0.047916,0.006429,0.063817,...,0.141614,0.001792,0.114526,0.055551,0.012903,0.012413,0.009278,0.026498,0.074783,0.000604
app_activity__add_to_basket_3d,0.009005,0.003625,0.000231,0.002255,0.005120,0.005223,0.047916,1.000000,0.001273,0.020987,...,0.005447,0.011766,0.062385,0.097396,0.108851,0.004858,0.003537,0.003710,0.029243,0.002635
app_activity__additional_1_3d,0.038830,0.015006,0.034796,0.039863,0.041960,0.047189,0.006429,0.001273,1.000000,0.023865,...,0.038438,0.165353,0.033256,0.010389,0.003832,0.001358,0.002502,0.005178,0.002332,0.003224
app_activity__additional_2_3d,0.041497,0.022386,0.020378,0.023909,0.037795,0.043996,0.063817,0.020987,0.023865,1.000000,...,0.031390,0.004400,0.052530,0.002820,0.000191,0.009973,0.007854,0.069543,0.021215,0.005612


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report
import joblib

# Load the dataset
df = pd.read_csv('interview_test_data.csv')

# Define features and target
X = df.drop(['click_event', 'click_time', 'imp_time'], axis=1)
y = df['click_event']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)])

# Train the model
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)[:, 1]

# Evaluate the model
print("ROC-AUC Score: ", roc_auc_score(y_test, y_pred_proba))
print("Classification Report: \n", classification_report(y_test, y_pred))

# Save the model for future use
joblib.dump(clf, 'adtech_model.pkl')

# Load the model (for future use)
# loaded_model = joblib.load('adtech_model.pkl')